In [2]:
import pandas as pd
from tqdm import tqdm
import pickle

In [56]:
df = pd.read_csv('data/zac2022_train_merged_final_preprocessing.json',keep_default_na=False)

In [ ]:
df.head(5)

In [60]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [ ]:
def new_sentences(row):
    from pyvi.ViTokenizer import tokenize
    title=row['title']
    text=row['text']
    return [tokenize(title),tokenize(text)]

In [ ]:
# shuffle df
df = df.sample(frac=1).reset_index(drop=True)
len(df)

In [ ]:
dff = df.parallel_apply(new_sentences,axis=1)

In [ ]:
sentences = dff.values.tolist()
len(sentences)

In [ ]:
# pickle.dump(sentences,open('data_gen/sentences.pkl','wb'))

In [3]:
sentences = pickle.load(open('data_gen/sentences.pkl','rb'))

In [ ]:
# sentences = []
# for i in tqdm(range(len(df))):
#     title = tokenize(df.iloc[i]['title'])
#     text = tokenize(df.iloc[i]['text'])
#     sentences.append( [title,text])

In [6]:
from sentence_transformers import SentenceTransformer, util
import torch

In [7]:
print(torch.cuda.is_available())

True


In [8]:
model = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base',device='cuda')

No sentence-transformers model found with name C:\Users\ngoph/.cache\torch\sentence_transformers\VoVanPhuc_sup-SimCSE-VietNamese-phobert-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\ngoph/.cache\torch\sentence_transformers\VoVanPhuc_sup-SimCSE-VietNamese-phobert-base were not used when initializing RobertaModel: ['mlp.dense.weight', 'mlp.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.device

In [5]:
import transformers
transformers.logging.set_verbosity_error()

In [ ]:
corpus_embeddings = model.encode(sentences,convert_to_tensor=True,show_progress_bar=True,device='cuda')

In [ ]:
# save corpus_embeddings
# pickle.dump(corpus_embeddings,open('data_gen/corpus_embeddings.pkl','wb'))

In [9]:
corpus_embeddings = pickle.load(open('data_gen/corpus_embeddings.pkl','rb'))

In [7]:
corpus_embeddings.shape

torch.Size([1273469, 768])

In [10]:
from pyvi.ViTokenizer import tokenize

In [12]:
from transformers import pipeline
qa = pipeline('question-answering', model='model_2\checkpoint-28391',device=0,)

In [61]:
df['title_tokenize'] = df['title'].parallel_apply(tokenize)

In [62]:
df.head()

,Unnamed: 0,id,url,title,text,timestamp,revid,title_tokenize
0,0,2,https://vi.wikipedia.org/wiki?curid=2,Trang Chính,Trang Chính __NOEDITSECTION__,2022-05-12T12:46:53Z,68591979,Trang_Chính
1,1,4,https://vi.wikipedia.org/wiki?curid=4,Internet Society,Internet Society Internet Society hay ISOC là ...,2022-01-20T07:59:10Z,67988747,Internet_Society
2,2,13,https://vi.wikipedia.org/wiki?curid=13,Tiếng Việt,"Tiếng Việt Tiếng Việt, cũng gọi là tiếng Việt ...",2022-05-29T03:42:42Z,68660631,Tiếng Việt
3,3,24,https://vi.wikipedia.org/wiki?curid=24,Ohio,"Ohio Ohio (viết tắt là OH, viết tắt cũ là O.) ...",2022-04-17T08:15:22Z,68482118,Ohio
4,4,26,https://vi.wikipedia.org/wiki?curid=26,California,"California California (phát âm như ""Ca-li-phót...",2022-06-16T15:27:07Z,68738039,California


In [63]:
def get_text(title):
    return df[df['title_tokenize']==title]['text'].values[0]

In [83]:
q_raw = '''Campuchia theo thể chế chính trị nào'''
q = tokenize(q_raw)
query_embedding = model.encode(q,convert_to_tensor=True,)
print(q_raw)
print(q)
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=10)[0]

answers =[]
for hit in hits:
    corpus_id = hit['corpus_id']
    doc_score = hit['score']

    title = sentences[corpus_id][0]
    text = get_text(title)
    answer = qa(question=q_raw,context=text)
    answer_score = answer['score']
    answers.append((title,answer['answer'],doc_score,answer_score))

sorted_answers = sorted(answers,key=lambda x:x[2]+x[3]*2,reverse=True)

for title,answer,doc_score,answer_score in sorted_answers:
    print('Title: ',title)
    print('Answer: ',answer)
    print('Doc score: ',doc_score)
    print('Answer score: ',answer_score)
    print('Calc score:',doc_score+answer_score*2)
    print('-----------------------')

Campuchia theo thể chế chính trị nào
Campuchia theo thể_chế chính_trị nào
Title:  Danh_sách chính_đảng Campuchia
Answer:  Đảng phái chính trị
Doc score:  0.7399005889892578
Answer score:  5.7283246966416676e-21
Calc score: 0.7399005889892578
-----------------------
Title:  Nhà_nước Campuchia
Answer:  Khmer Đỏ
Doc score:  0.7305138111114502
Answer score:  1.1103767611050832e-15
Calc score: 0.7305138111114524
-----------------------
Title:  Campuchia
Answer:  đền Angkor Wat
Doc score:  0.7113679647445679
Answer score:  0.0019612330943346024
Calc score: 0.7152904309332371
-----------------------
Title:  Đảng Trung_lập Khmer
Answer:  do Bou Hel và Ty Chhin thành lập vào năm 1993
Doc score:  0.7064779996871948
Answer score:  1.1868192952609599e-21
Calc score: 0.7064779996871948
-----------------------
Title:  Liên_minh Dân_chủ Campuchia
Answer:  Liên minh Dân chủ Campuchia Liên minh Dân chủ Campuchia
Doc score:  0.7007787823677063
Answer score:  1.3291574600931416e-13
Calc score: 0.70077878